In [1]:
import requests
import pandas as pd
from matplotlib import pyplot as plt
from scipy.stats import gamma
import numpy as np

In [2]:
import c3aidatalake

###Todo (by 11/3):
- Quantify government policy categories (LocationPolicySummary)
    - Bigger values for more restrictive policies
- Convert economic data to usable format
- Produce/visualize distributions and general statistics for data
    - Visualize dates of latest policies for states


In [3]:
policies = c3aidatalake.fetch(
    "locationpolicysummary", 
    {
        "spec" : {
            "limit" : -1
        }
    })

In [5]:
def printColValCounts(df, colName):
    print("Column Name: " + colName)
    print(df[colName].value_counts())

In [4]:
statePolicies = policies.drop(44)
statePolicies.columns

Index(['easingOrder', 'stayAtHome', 'mandatoryQuarantine',
       'nonEssentialBusiness', 'largeGatherings', 'schoolClosure',
       'restaurantLimit', 'barClosures', 'faceCoveringRequirement',
       'PrimaryElectionPostponement', 'emergencyDeclaration',
       'waiveTreatmentCost', 'freeVaccine',
       'waiverOfPriorAuthorizationRequirements', 'prescriptionRefill',
       'premiumPaymentGracePeriod', 'marketplaceSpecialEnrollmentPeriod',
       'section1135Waiver', 'paidSickLeaves',
       'expandsAccesstoTelehealthServices', 'id', 'lastSavedTimestamp',
       'numSavedVersions', 'savedVersion', 'location.id'],
      dtype='object')

In [6]:
for col in statePolicies:
    if not (col in quantifyDicts.keys()):
        print("Missing: " + col)
print("done")

Missing: waiveTreatmentCost
Missing: waiverOfPriorAuthorizationRequirements
Missing: prescriptionRefill
Missing: section1135Waiver
Missing: id
Missing: lastSavedTimestamp
Missing: numSavedVersions
Missing: savedVersion
Missing: location.id
done


Generating numerical columns from statePolicies:

Sora's Dictionaries:
'mandatoryQuarantine',
       'nonEssentialBusiness', 'largeGatherings', 'schoolClosure',
       'restaurantLimit', 'barClosures', 'faceCoveringRequirement',
       'PrimaryElectionPostponement', 'emergencyDeclaration',


In [5]:
quantifyDicts = {}
quantifyDicts["easingOrder"] = {
    "Reopened" : 0, 
    "Proceeding with Reopening" : 1,
    "Paused" : 2, 
    "New Restrictions Imposed" : 3
}
quantifyDicts["stayAtHome"] = {
    "No Action" : 0,
    "Lifted" : 0,
    "Rolled Back to High Risk Groups" : 1,
    "New Stay at Home Order" : 2,
    "Statewide" : 2
}
quantifyDicts["mandatoryQuarantine"]  = {
    "No Action" : 0,
    "Lifted" : 0,
    "From Certain States (New)" : 1,
    "Rolled Back to Certain States" : 1,
    "From Certain States" : 1,
    "Rolled Back to International Travel" : 2,
    "All Travelers" : 3
}
quantifyDicts["nonEssentialBusiness"] = {
    "No Action" : 0,
    "All Non-Essential Businesses Permitted to Reopen" : 0,
    "Some Non-Essential Businesses Permitted to Reopen" : 1,
    "All Non-Essential Businesses Permitted to Reopen with Reduced Capacity" : 1,
    "Some Non-Essential Businesses Permitted to Reopen with Reduced Capacity" : 2,
    "New Business Closures or Limits" : 3
}
quantifyDicts["largeGatherings"] = {
    "Lifted" : 0,
    "No Action" : 0,
    "Expanded to New Limit Above 25" : 1,
    "New Limit on Large Gatherings in Place" : 1,
    "Expanded to New Limit of 25 or Fewer" : 2,
    ">10 People Prohibited" : 3,
    "All Gatherings Prohibited" : 4
}
quantifyDicts["schoolClosure"] = {
    "Rescinded" : 0,
    "Recommended Closure for School Year" : 1,
    "Recommended Closure" : 2,
    "Closed for School Year" : 3,
    "Closed" : 4
}
quantifyDicts["restaurantLimit"] = {
    "No Action" : 0,
    "Reopened to Dine-in Service" : 1,
    "Reopened to Dine-in Service with Capacity Limits" : 2,
    "New Service Limits" : 3,
    "Newly Closed to Dine-in Service" : 3
}
quantifyDicts["barClosures"] = {
    "Reopened" : 0,
    "New Service Limits" : 1,
    "Closed" : 2,
    "Newly Closed" : 2
}
quantifyDicts["faceCoveringRequirement"] = {
    "No" : 0,
    "Required for Certain Employees" : 1,
    "Allows Local Officals to Require for General Public" : 1,
    "Required for Certain Employees; Allows Local Officials to Require for General Public" : 1,
    "Required for General Public" : 2
}
quantifyDicts["PrimaryElectionPostponement"] = {
    "No" : 0,
    "Postponed" : 1,
    "Canceled" : 2
}
quantifyDicts["emergencyDeclaration"] = {
    "Yes" : 0
}
quantifyDicts["waiveTreatmentCost"]  = {
    "No Action" : 0,
    "State-Insurer Agreement" : 1,
    "State Requires" : 2
}  
quantifyDicts["freeVaccine"] = {
    "No Action" : 0,
    "State-Insurer Agreement" : 1,
    "State Requires" : 2
}
quantifyDicts["waiverOfPriorAuthorizationRequirements"] = {
    "No  Action" : 0,
    "For COVID-19 Testing" : 1,
    "For COVID-19 Testing and Treatment" : 2
}
quantifyDicts["prescriptionRefill"] = {
    "No Action" : 0,
    "Expired" : 1,
    "State Requires" : 2
}
quantifyDicts["premiumPaymentGracePeriod"] = {
    "No Action" : 0,
    "Expired" : 1,
    "COVID-19 Diagnosis/Impacts Only" : 2,
    "Grace Period extended for All Individual Policies" : 3,
    "All Policies" : 4
}
quantifyDicts["marketplaceSpecialEnrollmentPeriod"] = {
    "No" : 0,
    "Ended" : 1,
    "Active" : 2
}
quantifyDicts["section1135Waiver"] = {
    "Unapproved" : 0,
    "Approved" : 1
}
quantifyDicts["paidSickLeaves"] = {
    "No Action" : 0,
    "Proposed - March 2020" : 1,
    "Enacted" : 2
}
quantifyDicts["expandsAccesstoTelehealthServices"] = {
    "no" : 0,
    "yes" : 1
}

In [8]:
def mapperGenerator(colName):
    
    def mapper(val):
        return quantifyDicts[colName][val]
    return mapper